In [3]:
import sympy as sy

import IPython
from IPython.display import display

import matplotlib.pyplot as plt

import numpy as np

from scipy.integrate import odeint

def displayH(a1,a2='', a3='', a4='', a5='', a6='', a7='',):
    latex_a1 = sy.latex(a1)
    latex_a2 = sy.latex(a2)
    latex_a3 = sy.latex(a3)
    latex_a4 = sy.latex(a4)
    latex_a5 = sy.latex(a5)
    latex_a6 = sy.latex(a6)
    latex_a7 = sy.latex(a7)
    display( IPython.core.display.Math(latex_a1 + latex_a2 + latex_a3 + latex_a4 + latex_a5 + latex_a6 + latex_a7)  )

$\newline$

## (3) Gravity-gradient stability simulation.
The linearized equations of motion for the gravity-gradient satellite derived in class
are as follows:

$$
\ddot{\theta} + 3n^2 \left( \frac{I_1 - I_3}{I_2} \right) \theta = 0 
\quad \text{...for the pitch angle}
$$

$$
\begin{pmatrix}
\dot{\phi} \\
\dot{\psi}
\end{pmatrix}
+ 
\begin{bmatrix}
0 & 0 \\
n(k_R - 1) & n(1 - k_Y)
\end{bmatrix}
\begin{pmatrix}
\phi \\
\psi
\end{pmatrix}
+
\begin{bmatrix}
4n^2 k_Y & 0 \\
0 & n^2 k_R
\end{bmatrix}
\begin{pmatrix}
\phi \\
\psi
\end{pmatrix}
= 
\begin{pmatrix}
0 \\
0
\end{pmatrix}
\quad \text{...for the roll-yaw angles}
$$

where,

$$
k_R = \frac{I_2 - I_1}{I_3}, \quad
k_Y = \frac{I_2 - I_3}{I_1}, \quad
n = \text{mean motion}
$$

A satellite is in a 7000 km circular orbit around the Earth ($\mu = 398600.4418 \, \text{km}^3/\text{s}^2$).The initial conditions for the Euler angles and the Euler angle rates are:

$(\phi_0, \theta_0, \psi_0) = (30^\circ, 20^\circ, 10^\circ) \quad \text{and} \quad (\dot{\phi}_0, \dot{\theta}_0, \dot{\psi}_0) = (0, 0, 0) \, \text{rad/s}$.

Further, the following three sets of initial conditions for the moment of inertia matrix are given:

$$
I_{I} = 
\begin{bmatrix}
I_1 & 0 & 0 \\
0 & I_2 & 0 \\
0 & 0 & I_3
\end{bmatrix}
=
\begin{bmatrix}
I_{\text{BIG}} & 0 & 0 \\
0 & I_{\text{Inter}} & 0 \\
0 & 0 & I_{\text{small}}
\end{bmatrix}
$$

$$
I_{II} = 
\begin{bmatrix}
I_1 & 0 & 0 \\
0 & I_2 & 0 \\
0 & 0 & I_3
\end{bmatrix}
=
\begin{bmatrix}
I_{\text{Inter}} & 0 & 0 \\
0 & I_{\text{BIG}} & 0 \\
0 & 0 & I_{\text{small}}
\end{bmatrix}
$$

$$
I_{III} = 
\begin{bmatrix}
I_1 & 0 & 0 \\
0 & I_2 & 0 \\
0 & 0 & I_3
\end{bmatrix}
=
\begin{bmatrix}
I_{\text{small}} & 0 & 0 \\
0 & I_{\text{Inter}} & 0 \\
0 & 0 & I_{\text{BIG}}
\end{bmatrix}
$$

where,

$$
I_{\text{BIG}} = 400 \, \text{kg m}^2, \\
I_{\text{Inter}} = 300 \, \text{kg m}^2, \\
I_{\text{small}} = 200 \, \text{kg m}^2.
$$

In [53]:
mu = 398600.4418
a = 7000
x0 = np.radians([30,20,10,0,0,0])

I_big = 400
I_inter = 300
I_small = 200

I1 = [I_big,I_inter,I_small]
I2 = [I_inter,I_big,I_small]
I3 = [I_small,I_inter,I_big]

def gravity_gradient(x,t,mu,a,I):
    n = (mu/a**3)**0.5
    k_R = (I[1]-I[0])/I[2]
    k_Y = (I[1]-I[2])/I[0]
    return [x[3],
            x[4],
            x[5],
            -n*(1-k_Y)*x[5]-4*n**2*k_Y*x[0],
            -3*n**2*(I[0]-I[2])/I[1]*x[1],
            -n*(k_R-1)*x[3]-n**2*k_R*x[2]]

times = np.arange(0,10000,0.1)
tol = 1e-12

x1 = np.degrees(odeint(gravity_gradient,x0,times,rtol=tol,atol=tol,args=(mu,a,I1)).T)
x2 = np.degrees(odeint(gravity_gradient,x0,times,rtol=tol,atol=tol,args=(mu,a,I2)).T)
x3 = np.degrees(odeint(gravity_gradient,x0,times,rtol=tol,atol=tol,args=(mu,a,I3)).T)

In [ ]:
fig,axes = plt.subplots(6,3,sharex=True,figsize=(5,3))
axes[0,1].set_title(r"$\vec{x}$ vs Time")
for i in range(6):
    for j,x in enumerate([x1,x2,x3]):
        if i < 3:
            axes[i,j].plot(times,x[i]%360)
        else:
            axes[i,j].plot(times,x[i])

axes[0,0].set_ylabel(r"$\phi(t)$")
axes[1,0].set_ylabel(r"$\theta(t)$")
axes[2,0].set_ylabel(r"$\psi(t)$")
axes[3,0].set_ylabel(r"$\dot{\phi}(t)$")
axes[4,0].set_ylabel(r"$\dot{\theta}(t)$")
axes[5,0].set_ylabel(r"$\dot{\psi}(t)$")
axes[5,0].set_xlabel("time [s]")
plt.show()